In [1]:
from pymongo import MongoClient

In [2]:
mongo_url = "mongodb://localhost/companies.json"

In [3]:
client = MongoClient (mongo_url)

In [4]:
db = client.get_database()

In [5]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [6]:
#Almaceno todos los paises que tienen country_code
cc = db.companies.find({"offices.country_code":{"$ne": None}})

In [7]:
latcc = db.companies.find({"$and":[{"offices.country_code":{"$ne": None}}, {"offices.longitud":{"$ne": None}},{"offices.latitude":{"$ne": None}}]})

In [8]:
len(list(latcc))

5057

In [9]:
#Vemos la longitud de la lista de las empresas que tienen lat, long y country_code not null

In [10]:
res = db.companies.aggregate([
                      {"$unwind":"$offices" },
                      {"$match":{"offices.latitude":{"$ne":None}}},
                      {"$match":{"offices.longitude":{"$ne":None}}},
                      {"$project":{"_id":0}}
                       ])

In [11]:
db.companies_unwinded.insert_many(res)

In [12]:
offices = db.companies_unwinded

In [13]:
offs = offices.find({},{"offices":1})

In [14]:
for r in offs:
    filt = {"_id":r["_id"]}
    coord = {
              "type": "Point",
              "coordinates": [r["offices"]["longitude"], r["offices"]["latitude"]]
            }
    update = {"$set":{"coord":coord}}
    offices.update_one(filt,update)